In [15]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) yes
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your term

In [3]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [18]:
from datasets import load_dataset,DatasetDict

# Load the dataset
dataset = load_dataset('snli')

# Select training, validation, and testing samples
train_samples = dataset['train'].select([i * 550 for i in range(1000)])
validation_samples = dataset['validation'].select([i * 100 for i in range(100)])
test_samples = dataset['test'].select([i * 100 for i in range(100)])

# Define a function to transform the data to the required format
def transform_example(example, is_test=False):
    premise = example['premise'].strip()
    hypothesis = example['hypothesis'].strip()
    label = example['label'] if not is_test else None

    # Apply the new format template based on dataset type (train/validation vs. test)
    if is_test:
        formatted_text = (f" PREMISE: {premise} HYPOTHESIS: {hypothesis} "
                          f"Based on the above information, does the hypothesis entail, contradict, "
                          f"or is it neutral with respect to the premise? "
                          f"Please classify as: 'Entailment' (0), 'Neutral' (1), or 'Contradiction' (2).")
    else:
        formatted_text = (f"PREMISE: {premise} HYPOTHESIS: {hypothesis} "
                          f"Based on the above information, does the hypothesis entail, contradict, "
                          f"or is it neutral with respect to the premise? "
                          f"Please classify as: 'Entailment' (0), 'Neutral' (1), or 'Contradiction' (2)."
                          f"LABEL: {label}")

    return {'text': formatted_text}

# Apply the transformation on training, validation, and testing samples
transformed_train = train_samples.map(lambda x: transform_example(x, is_test=False))
transformed_validation = validation_samples.map(lambda x: transform_example(x, is_test=False))
transformed_test = test_samples.map(lambda x: transform_example(x, is_test=True))

# Display some examples to verify
print(transformed_train[0])  # Example from the training set with label
print(transformed_validation[0])  # Example from the validation set with label
print(transformed_test[0])  # Example from the test set without label

transformed_dataset = DatasetDict({
    'train': transformed_train,
    'validation': transformed_validation,
    'test': transformed_test
})

# Push to Hugging Face Hub
transformed_dataset.push_to_hub("Mahisha268/snli-phi-transformed", private=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1, 'text': "PREMISE: A person on a horse jumps over a broken down airplane. HYPOTHESIS: A person is training his horse for a competition. Based on the above information, does the hypothesis entail, contradict, or is it neutral with respect to the premise? Please classify as: 'Entailment' (0), 'Neutral' (1), or 'Contradiction' (2).LABEL: 1"}
{'premise': 'Two women are embracing while holding to go packages.', 'hypothesis': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', 'label': 1, 'text': "PREMISE: Two women are embracing while holding to go packages. HYPOTHESIS: The sisters are hugging goodbye while holding to go packages after just eating lunch. Based on the above information, does the hypothesis entail, contradict, or is it neutral with respect to the premise? Please classify as: 'Entailment' (0), '

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/695 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Mahisha268/snli-phi-transformed/commit/d2c6e22d4e358f8b7806b324f12597d5cc66fe08', commit_message='Upload dataset', commit_description='', oid='d2c6e22d4e358f8b7806b324f12597d5cc66fe08', pr_url=None, pr_revision=None, pr_num=None)